In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [ ]:
normlized_beyond_mead_data = pd.read_csv('normalized_sampled_data_beyond_meat.csv')

In [ ]:
feature_cols = normlized_beyond_mead_data.columns[normlized_beyond_mead_data.columns.str.startswith('feature')]

X_train = normlized_beyond_mead_data[feature_cols].to_numpy()
Y_train = normlized_beyond_mead_data.target.to_numpy()

X_train, Y_train = shuffle(X_train, Y_train)

X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.1)

In [ ]:
def defineNN_regression(n_inputFeatures):
    activation = "relu"
    regularizationConst_l1 = 0.00000#3
    regularizationConst_l2 = 0.00000#3
    size = 512
    X_input = Input(shape=(n_inputFeatures,))
    X = Dropout(0.00, input_shape = (n_inputFeatures,))(X_input)
 
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    #X = Dropout(dropoutRate, input_shape = (size,))(X)
    #X = BatchNormalization(axis = -1)(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)

    X = Dense(size/4, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size/8, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size/16, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)
    X = Dense(size/32, activation=activation, kernel_regularizer=regularizers.l1_l2(l1=regularizationConst_l1, l2=regularizationConst_l2), bias_regularizer=regularizers.l2(regularizationConst_l2), activity_regularizer=regularizers.l2(regularizationConst_l2))(X)

    X = Dense(1, activation="linear")(X)
    
    model = Model(inputs = X_input, outputs = X, name='deepNN')

    return model

In [ ]:
def absError(target, pred):
  """Calculates how often the max prediction matches one-hot labels."""

  return tf.reduce_sum(tf.math.abs(target-pred))

In [ ]:
modelNN_reg = defineNN_regression(X_train.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.99)

METRICS = [
    #absError, 
    tf.keras.losses.MeanAbsoluteError(),
]
modelNN_reg.compile(optimizer=optAdam, loss='mean_squared_error', metrics=METRICS)
modelNN_reg.load_weights("first_0049train_0048test_0027val.h5")

In [ ]:
history = modelNN_reg.fit(X_train, Y_train, epochs = 50, batch_size = 256*128*4, validation_data=(X_test, Y_test))


In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
#print(history.history)
fig = plt.figure()
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
modelNN_reg.save_weights("first_0049train_0048test_0027val.h5")

In [ ]:
normlized_tesla_data = pd.read_csv('normalized_sampled_data_tesla.csv')

In [ ]:
def evaluatePerformance(normalized_data):
    X_val = normalized_data[feature_cols].to_numpy()
    Y_val = np.reshape(normalized_data.target.to_numpy(), (X_val.shape[0], 1))

    pred = modelNN_reg.predict(X_val)
    normalizedValue = np.reshape(normalized_data['normalizeValue'].to_numpy(), (pred.shape[0], 1))
    
    
    Y_val_abs = np.multiply(Y_val, normalizedValue)
    pred_abs  = np.multiply(pred, normalizedValue)
    
    absError = np.sum(np.abs(Y_val_abs - pred_abs))
    relError = np.sum(np.abs(Y_val - pred))
    print(absError, relError)
    print(absError/len(pred_abs), relError/len(pred_abs))
    
    timestamps = np.linspace(0, pred_abs.shape[0]-1, pred_abs.shape[0])
    
    index = 200
    delta = 200
#     plt.subplot(211)
#     plt.plot(timestamps[index:index + delta], pred_abs[index:index + delta], 'r--')
#     plt.plot(timestamps[index:index + delta], Y_val_abs[index:index + delta], 'b-')
#     plt.legend(['prediction', 'groundTruth'])
#     plt.xlabel('timestamps')
#     plt.ylabel('value')
#     plt.show()
    
    #plt.subplot(212)
    fig = plt.figure()
    plt.plot(timestamps, pred_abs, 'r--')
    plt.plot(timestamps, Y_val_abs, 'b-')
    plt.legend(['prediction', 'groundTruth'])
    plt.xlabel('timestamps')
    plt.ylabel('value')
    plt.show()

In [ ]:
evaluatePerformance(normlized_beyond_mead_data)

In [ ]:
evaluatePerformance(normlized_tesla_data)